# Script behaves decently well, should be cleaned to be made more modular/robust
## TODOs
* include xpath fetching in helper methods to resolve issues of stale webelements
* fix conditional statement for clicking on ellipsis
* convert driver loop into function


Once these todos are accomplished, the script will be importable, allowing for new scraping of the website. This could lead to a sufficiently large corpus to tune an LLM

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [26]:
driver = webdriver.Chrome()

In [44]:
driver.get("https://cgmix.uscg.mil/IIR/IIRSearch.aspx")

In [45]:
begin_date_xpath = '//*[@id="TextBoxFromDate"]'

begin_date = driver.find_element(By.XPATH, begin_date_xpath)

end_date_xpath = '//*[@id="TextBoxToDate"]'
end_date = driver.find_element(By.XPATH, end_date_xpath)

In [46]:
vessel_service_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[4]/td[2]/select"
vessel_service = driver.find_element(By.XPATH, vessel_service_xpath)

involved_org_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[6]/td[2]/input"
involved_org = driver.find_element(By.XPATH, involved_org_xpath)

In [47]:
kword_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[8]/td[2]/input"

kword = driver.find_element(By.XPATH, kword_xpath)

search_btn_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div/table/tbody/tr[13]/td/input[1]'
search_btn = driver.find_element(By.XPATH, search_btn_xpath)

include xpath fetching in below helpers

In [31]:
def set_begin_date(date):
    begin_date.clear()
    begin_date.send_keys(date)

def set_end_date(date):
    end_date.clear()
    end_date.send_keys(date)

def set_involved_org(org):
    involved_org.clear()
    involved_org.send_keys(org)

def set_vessel_service(service):
    vessel_service.send_keys(service)

def set_kword(kw):
    kword.clear()
    kword.send_keys(kw)

In [48]:
set_begin_date("01/01/2020")
set_vessel_service("Freight Ship")
search_btn.click()

turn below into a method

In [49]:
table_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody'
table = driver.find_element(By.XPATH, table_xpath)

trs = table.find_elements(By.TAG_NAME, 'tr')

In [50]:
def gen_links(trs):
    details_links = []
    for i, tr in enumerate(trs[1:]):
        try:
            tds = tr.find_elements(By.TAG_NAME, 'td')
            details_link = tds[0].find_element(By.TAG_NAME, 'a')
            title = tds[1].text
            details_links.append((title, details_link))
        except:
            print("failed on " + str(i))
            details_links.append(tds)
    return details_links

In [ ]:
gen_links(trs)

In [52]:
def refresh_links(driver):
    table_xpath = '/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody'
    table = driver.find_element(By.XPATH, table_xpath)

    trs = table.find_elements(By.TAG_NAME, 'tr')
    details_links = gen_links(trs[:11])
    return details_links

In [39]:
def process_page(driver, qa=False):
    from time import sleep
    from random import randint
    sleep(1)
    div_xpath = "/html/body/form/div[3]/table/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/div"
    div = driver.find_element(By.XPATH, div_xpath)
    tbls = div.find_elements(By.TAG_NAME, 'table')
    pg_tables =[]
    for tbl in tbls:
        # save tbl text to string
        s = tbl.text
        pg_tables.append(s)
        # print(tbl.text)
    if qa:
        # need to start at top of page
        driver.execute_script("window.scrollTo(0, 0)")
        # need to loop over length of page
        # can I get the length of the page from the driver object?
        ht = driver.execute_script("return document.body.scrollHeight")
        for i in range(0, ht, 100):
            driver.execute_script("window.scrollTo(0, " + str(i) + ")")
            driver.save_screenshot(str(i) +"_screenshot.png")
        
    # return to previous page
    driver.back()
    return pg_tables


In [ ]:
import pickle
import tqdm
ctr = 11
progress = tqdm.tqdm(1660)
while True:
    print("Scraping page " + str(ctr))
    pgs = []
    for i in range(10):
        links = refresh_links(driver)
        links[i][1].click()
        title = ["TITLE: "+links[i][0]]
        pg = process_page(driver)
        pgs.append(title + pg)
        progress.update(1)
    print("-"*5 + "saving page " + str(ctr) + "-"*5)
    with open("2020_Fship/page_" + str(ctr) + ".pkl", "wb") as f:
        pickle.dump(pgs, f)
    ctr += 1
    pg_bar_xpath = "/html/body/form/div[3]/div/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td/div/table/tbody/tr[5]/td/div/table/tbody/tr[12]/td/table/tbody/tr"
    pg_bar = driver.find_element(By.XPATH, pg_bar_xpath)
    nxt_pg = pg_bar.find_elements(By.TAG_NAME, "td")
    target = [x for x in nxt_pg if x.text == str(ctr)]
    
    # if we can't find the page number, we need to find the ellipsis
    if len(target) == 0:
        target = [x for x in nxt_pg if x.text == "..."]
        # for the first 10, there is only one ellipsis
        if len(target) == 1:
            target = target[0]
        # for the rest, there are two
        else:
            target = target[1]
    # Logic is faulty, script will not stop!!!
            # target is assigned above, and then referenced below
            # changing the expected behavior
    else:
        target = target[0]
    
    target.click()


# Cell below was a quick fix to get the last page of data, see above comments regarding failure towards end of website content

In [ ]:
# for i in range(10):
#     links = refresh_links(driver)
#     links[i][1].click()
#     title = ["TITLE: "+links[i][0]]
#     print(title[0])
#     pg = process_page(driver)
#     pgs.append(title + pg)
#     progress.update(1)
# print("-"*5 + "saving page " + "-"*5)
# with open("2020_Fship/page_167" + ".pkl", "wb") as f:
#     pickle.dump(pgs, f)